# Day 3 - MCP Integration with LangGraph
# MCP(Model Context Protocol) 통합 실습

이 실습에서는 LangGraph와 Day 1의 파인튜닝 모델을 사용하여
MCP(Model Context Protocol)를 통한 외부 도구 통합 시스템을 구현합니다.

## 핵심 개념
- MCP(Model Context Protocol) 이해
- 외부 API 및 도구 통합
- 도구 선택 및 실행 워크플로
- 결과 통합 및 응답 생성
- 에러 처리 및 백업 전략

In [ ]:
!pip install langchain langgraph requests beautifulsoup4 transformers torch python-dateutil wikipedia-api yfinance

In [ ]:
import os
import json
import requests
from typing import List, Dict, Any, Optional, TypedDict, Callable
from dataclasses import dataclass
from datetime import datetime, timedelta
import re

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

from langchain.schema import Document
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor

from bs4 import BeautifulSoup
import wikipedia
import yfinance as yf
from dateutil import parser

print("라이브러리 import 완료")

## 1. Day 1 파인튜닝 모델 로드

In [ ]:
class Day1FinetunedLLM:
    def __init__(self, model_name="ryanu/my-exaone-raft-model"):
        self.model_name = model_name
        self.base_model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"모델 로딩 중: {model_name}")
        print(f"사용 디바이스: {self.device}")
        
        # 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.base_model,
            trust_remote_code=True
        )
        
        # 베이스 모델 로드
        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        # 파인튜닝된 어댑터 로드
        try:
            self.model = PeftModel.from_pretrained(self.model, model_name)
            print("✅ 파인튜닝 모델 로드 완료")
        except Exception as e:
            print(f"⚠️ 파인튜닝 모델 로드 실패, 베이스 모델 사용: {e}")
    
    def generate(self, prompt: str, max_length: int = 512, temperature: float = 0.7) -> str:
        """텍스트 생성"""
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=2048
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=inputs['input_ids'].shape[1] + max_length,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        generated_text = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return generated_text.strip()

# 모델 초기화
llm = Day1FinetunedLLM()

## 2. MCP 도구 정의

다양한 외부 서비스와 API를 통합하는 도구들을 정의합니다.

In [ ]:
@dataclass
class MCPTool:
    """MCP 도구 기본 클래스"""
    name: str
    description: str
    parameters: Dict[str, Any]
    function: Callable

class MCPToolRegistry:
    """MCP 도구 레지스트리"""
    
    def __init__(self):
        self.tools = {}
        self._register_default_tools()
    
    def register_tool(self, tool: MCPTool):
        """도구 등록"""
        self.tools[tool.name] = tool
    
    def get_tool(self, name: str) -> Optional[MCPTool]:
        """도구 가져오기"""
        return self.tools.get(name)
    
    def list_tools(self) -> List[str]:
        """등록된 도구 목록"""
        return list(self.tools.keys())
    
    def get_tools_description(self) -> str:
        """도구 설명 텍스트"""
        descriptions = []
        for name, tool in self.tools.items():
            param_desc = ", ".join([f"{k}: {v.get('description', v.get('type', ''))}" 
                                    for k, v in tool.parameters.items()])
            descriptions.append(f"{name}: {tool.description} (매개변수: {param_desc})")
        return "\n".join(descriptions)
    
    def _register_default_tools(self):
        """기본 도구들 등록"""
        
        # 웹 검색 도구
        def web_search(query: str, max_results: int = 5) -> Dict[str, Any]:
            """웹 검색 (간단한 구현)"""
            try:
                # 실제로는 Google Search API나 다른 검색 API 사용
                # 여기서는 시뮬레이션
                results = [
                    {
                        "title": f"검색 결과 {i+1}: {query}",
                        "url": f"https://example.com/result{i+1}",
                        "snippet": f"{query}에 대한 정보를 제공하는 웹페이지입니다."
                    }
                    for i in range(min(max_results, 3))
                ]
                return {"success": True, "results": results}
            except Exception as e:
                return {"success": False, "error": str(e)}
        
        self.register_tool(MCPTool(
            name="web_search",
            description="웹에서 정보를 검색합니다",
            parameters={
                "query": {"type": "string", "description": "검색할 키워드"},
                "max_results": {"type": "integer", "description": "최대 결과 수", "default": 5}
            },
            function=web_search
        ))
        
        # 위키피디아 검색 도구
        def wikipedia_search(query: str, lang: str = "ko") -> Dict[str, Any]:
            """위키피디아 검색"""
            try:
                wikipedia.set_lang(lang)
                # 검색 결과 가져오기
                search_results = wikipedia.search(query, results=3)
                
                if not search_results:
                    return {"success": False, "error": "검색 결과를 찾을 수 없습니다"}
                
                # 첫 번째 결과의 요약 가져오기
                try:
                    summary = wikipedia.summary(search_results[0], sentences=3)
                    page = wikipedia.page(search_results[0])
                    
                    return {
                        "success": True,
                        "title": page.title,
                        "summary": summary,
                        "url": page.url,
                        "related_topics": search_results[1:]
                    }
                except wikipedia.DisambiguationError as e:
                    return {
                        "success": True,
                        "title": f"여러 의미: {query}",
                        "summary": f"'{query}'는 여러 의미를 가집니다.",
                        "options": e.options[:5],
                        "related_topics": e.options[:3]
                    }
                except Exception as e:
                    return {"success": False, "error": f"페이지 로딩 실패: {str(e)}"}
                    
            except Exception as e:
                return {"success": False, "error": str(e)}
        
        self.register_tool(MCPTool(
            name="wikipedia_search",
            description="위키피디아에서 정보를 검색합니다",
            parameters={
                "query": {"type": "string", "description": "검색할 주제"},
                "lang": {"type": "string", "description": "언어 코드", "default": "ko"}
            },
            function=wikipedia_search
        ))
        
        # 주식 정보 조회 도구
        def get_stock_info(symbol: str, period: str = "1d") -> Dict[str, Any]:
            """주식 정보 조회"""
            try:
                stock = yf.Ticker(symbol)
                hist = stock.history(period=period)
                info = stock.info
                
                if hist.empty:
                    return {"success": False, "error": f"주식 정보를 찾을 수 없습니다: {symbol}"}
                
                latest = hist.iloc[-1]
                
                return {
                    "success": True,
                    "symbol": symbol,
                    "company_name": info.get('longName', symbol),
                    "current_price": round(latest['Close'], 2),
                    "change": round(latest['Close'] - latest['Open'], 2),
                    "change_percent": round((latest['Close'] - latest['Open']) / latest['Open'] * 100, 2),
                    "volume": int(latest['Volume']),
                    "high": round(latest['High'], 2),
                    "low": round(latest['Low'], 2),
                    "market_cap": info.get('marketCap'),
                    "sector": info.get('sector', 'N/A')
                }
                
            except Exception as e:
                return {"success": False, "error": str(e)}
        
        self.register_tool(MCPTool(
            name="get_stock_info",
            description="주식 정보를 조회합니다",
            parameters={
                "symbol": {"type": "string", "description": "주식 심볼 (예: AAPL, GOOGL, 005930.KS)"},
                "period": {"type": "string", "description": "기간 (1d, 5d, 1mo, 3mo, 1y)", "default": "1d"}
            },
            function=get_stock_info
        ))
        
        # 날씨 정보 도구 (시뮬레이션)
        def get_weather(city: str, units: str = "metric") -> Dict[str, Any]:
            """날씨 정보 조회 (시뮬레이션)"""
            # 실제로는 OpenWeatherMap API 등을 사용
            weather_data = {
                "서울": {"temp": 15, "humidity": 60, "description": "맑음"},
                "부산": {"temp": 18, "humidity": 70, "description": "구름 많음"},
                "제주": {"temp": 20, "humidity": 80, "description": "비"},
            }
            
            if city in weather_data:
                data = weather_data[city]
                return {
                    "success": True,
                    "city": city,
                    "temperature": data["temp"],
                    "humidity": data["humidity"],
                    "description": data["description"],
                    "units": units
                }
            else:
                return {
                    "success": True,
                    "city": city,
                    "temperature": 22,
                    "humidity": 65,
                    "description": "맑음",
                    "units": units
                }
        
        self.register_tool(MCPTool(
            name="get_weather",
            description="특정 도시의 날씨 정보를 조회합니다",
            parameters={
                "city": {"type": "string", "description": "도시명"},
                "units": {"type": "string", "description": "온도 단위 (metric, imperial)", "default": "metric"}
            },
            function=get_weather
        ))
        
        # 계산기 도구
        def calculate(expression: str) -> Dict[str, Any]:
            """수학 계산"""
            try:
                # 보안을 위해 허용된 문자만 사용
                allowed_chars = set('0123456789+-*/.() ')
                if not all(c in allowed_chars for c in expression):
                    return {"success": False, "error": "허용되지 않은 문자가 포함되어 있습니다"}
                
                result = eval(expression)
                return {
                    "success": True,
                    "expression": expression,
                    "result": result
                }
            except Exception as e:
                return {"success": False, "error": str(e)}
        
        self.register_tool(MCPTool(
            name="calculate",
            description="수학 계산을 수행합니다",
            parameters={
                "expression": {"type": "string", "description": "계산할 수학 표현식"}
            },
            function=calculate
        ))

# MCP 도구 레지스트리 초기화
mcp_registry = MCPToolRegistry()
print("✅ MCP 도구 레지스트리 초기화 완료")
print(f"등록된 도구: {mcp_registry.list_tools()}")

## 3. MCP 상태 정의

In [ ]:
class MCPState(TypedDict):
    """MCP 통합 워크플로 상태"""
    user_query: str                    # 사용자 질문
    intent_analysis: Dict[str, Any]    # 의도 분석 결과
    selected_tools: List[str]          # 선택된 도구들
    tool_calls: List[Dict[str, Any]]   # 도구 호출 정보
    tool_results: List[Dict[str, Any]] # 도구 실행 결과
    aggregated_info: str               # 통합된 정보
    final_response: str                # 최종 응답
    confidence: float                  # 신뢰도
    errors: List[str]                  # 오류 목록
    retry_count: int                   # 재시도 횟수

print("✅ MCP 상태 클래스 정의 완료")

## 4. MCP 워크플로 노드들

In [ ]:
def analyze_intent(state: MCPState) -> MCPState:
    """사용자 의도 분석 및 필요한 도구 식별"""
    user_query = state["user_query"]
    
    print(f"🧠 의도 분석: {user_query}")
    
    # 의도 분석 프롬프트
    tools_description = mcp_registry.get_tools_description()
    
    prompt = f"""다음 사용자 질문을 분석하고 필요한 도구들을 선택해주세요.

사용자 질문: {user_query}

사용 가능한 도구들:
{tools_description}

분석 결과를 다음 형식으로 제공해주세요:
의도: [질문의 주요 의도]
필요한 도구: [도구명1, 도구명2, ...]
이유: [도구 선택 이유]

분석 결과:"""
    
    analysis_text = llm.generate(prompt, max_length=300, temperature=0.3)
    
    # 간단한 키워드 기반 도구 선택 (백업)
    selected_tools = []
    query_lower = user_query.lower()
    
    if any(word in query_lower for word in ['검색', '찾아', '알려', '정보']):
        if any(word in query_lower for word in ['위키', '백과']):
            selected_tools.append('wikipedia_search')
        else:
            selected_tools.append('web_search')
    
    if any(word in query_lower for word in ['주식', '주가', '증권', 'stock']):
        selected_tools.append('get_stock_info')
    
    if any(word in query_lower for word in ['날씨', 'weather']):
        selected_tools.append('get_weather')
    
    if any(word in query_lower for word in ['계산', '더하기', '빼기', '곱하기', '나누기', '+', '-', '*', '/']):
        selected_tools.append('calculate')
    
    # 기본적으로 검색 도구 포함
    if not selected_tools:
        selected_tools.append('web_search')
    
    intent_analysis = {
        "analysis_text": analysis_text,
        "detected_keywords": [word for word in ['검색', '주식', '날씨', '계산', '위키'] if word in query_lower],
        "query_type": "informational" if any(word in query_lower for word in ['무엇', '어떻게', '왜']) else "transactional"
    }
    
    print(f"📊 선택된 도구: {selected_tools}")
    
    return {
        **state,
        "intent_analysis": intent_analysis,
        "selected_tools": selected_tools,
        "retry_count": state.get("retry_count", 0),
        "errors": state.get("errors", [])
    }

def prepare_tool_calls(state: MCPState) -> MCPState:
    """도구 호출 매개변수 준비"""
    user_query = state["user_query"]
    selected_tools = state["selected_tools"]
    
    print(f"🛠 도구 호출 준비: {selected_tools}")
    
    tool_calls = []
    
    for tool_name in selected_tools:
        tool = mcp_registry.get_tool(tool_name)
        if not tool:
            continue
        
        # 도구별 매개변수 추출
        if tool_name == "web_search":
            tool_calls.append({
                "tool": tool_name,
                "parameters": {
                    "query": user_query,
                    "max_results": 5
                }
            })
        
        elif tool_name == "wikipedia_search":
            # 질문에서 검색어 추출 (간단한 방식)
            search_query = user_query.replace('위키피디아에서', '').replace('검색해', '').strip()
            tool_calls.append({
                "tool": tool_name,
                "parameters": {
                    "query": search_query,
                    "lang": "ko"
                }
            })
        
        elif tool_name == "get_stock_info":
            # 주식 심볼 추출 시도
            symbols = []
            query_upper = user_query.upper()
            
            # 일반적인 주식 심볼들
            common_symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', '005930.KS', '000660.KS']
            for symbol in common_symbols:
                if symbol in query_upper or symbol.replace('.KS', '') in query_upper:
                    symbols.append(symbol)
            
            # 한국 주식명 매핑
            korean_stocks = {
                '삼성전자': '005930.KS',
                '애플': 'AAPL',
                '구글': 'GOOGL',
                '테슬라': 'TSLA',
                '마이크로소프트': 'MSFT',
                '아마존': 'AMZN'
            }
            
            for korean_name, symbol in korean_stocks.items():
                if korean_name in user_query:
                    symbols.append(symbol)
            
            # 기본값
            if not symbols:
                symbols = ['AAPL']  # 기본으로 Apple 주식
            
            for symbol in symbols[:1]:  # 첫 번째만 사용
                tool_calls.append({
                    "tool": tool_name,
                    "parameters": {
                        "symbol": symbol,
                        "period": "1d"
                    }
                })
        
        elif tool_name == "get_weather":
            # 도시명 추출
            cities = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '제주']
            detected_city = None
            
            for city in cities:
                if city in user_query:
                    detected_city = city
                    break
            
            if not detected_city:
                detected_city = "서울"  # 기본값
            
            tool_calls.append({
                "tool": tool_name,
                "parameters": {
                    "city": detected_city,
                    "units": "metric"
                }
            })
        
        elif tool_name == "calculate":
            # 수식 추출 (간단한 방식)
            import re
            # 숫자와 연산자 패턴 찾기
            math_pattern = r'[0-9+\-*/().\s]+'
            matches = re.findall(math_pattern, user_query)
            
            expression = None
            for match in matches:
                if any(op in match for op in ['+', '-', '*', '/']):
                    expression = match.strip()
                    break
            
            if not expression:
                expression = "2 + 2"  # 기본값
            
            tool_calls.append({
                "tool": tool_name,
                "parameters": {
                    "expression": expression
                }
            })
    
    print(f"📋 준비된 도구 호출: {len(tool_calls)}개")
    
    return {
        **state,
        "tool_calls": tool_calls
    }

def execute_tools(state: MCPState) -> MCPState:
    """도구 실행"""
    tool_calls = state["tool_calls"]
    
    print(f"⚡ 도구 실행 중: {len(tool_calls)}개")
    
    tool_results = []
    errors = list(state.get("errors", []))
    
    for call in tool_calls:
        tool_name = call["tool"]
        parameters = call["parameters"]
        
        print(f"🔧 실행 중: {tool_name} with {parameters}")
        
        try:
            tool = mcp_registry.get_tool(tool_name)
            if tool:
                result = tool.function(**parameters)
                tool_results.append({
                    "tool": tool_name,
                    "parameters": parameters,
                    "result": result,
                    "success": result.get("success", True)
                })
                
                if result.get("success", True):
                    print(f"✅ {tool_name} 성공")
                else:
                    print(f"❌ {tool_name} 실패: {result.get('error', 'Unknown error')}")
                    errors.append(f"{tool_name}: {result.get('error', 'Unknown error')}")
            else:
                error_msg = f"도구를 찾을 수 없음: {tool_name}"
                errors.append(error_msg)
                print(f"❌ {error_msg}")
                
        except Exception as e:
            error_msg = f"{tool_name} 실행 오류: {str(e)}"
            errors.append(error_msg)
            print(f"❌ {error_msg}")
            
            tool_results.append({
                "tool": tool_name,
                "parameters": parameters,
                "result": {"success": False, "error": str(e)},
                "success": False
            })
    
    print(f"📊 실행 완료: {len([r for r in tool_results if r['success']])}/{len(tool_results)} 성공")
    
    return {
        **state,
        "tool_results": tool_results,
        "errors": errors
    }

def aggregate_information(state: MCPState) -> MCPState:
    """도구 결과 통합"""
    user_query = state["user_query"]
    tool_results = state["tool_results"]
    
    print("📊 정보 통합 중...")
    
    # 성공한 결과들만 통합
    successful_results = [r for r in tool_results if r["success"]]
    
    if not successful_results:
        aggregated_info = "죄송합니다. 요청하신 정보를 가져오는데 실패했습니다."
        confidence = 0.1
    else:
        # 결과 요약 생성
        info_parts = []
        
        for result in successful_results:
            tool_name = result["tool"]
            data = result["result"]
            
            if tool_name == "web_search":
                if data.get("results"):
                    info_parts.append(f"웹 검색 결과: {len(data['results'])}개 결과를 찾았습니다.")
                    for i, res in enumerate(data["results"][:2], 1):
                        info_parts.append(f"  {i}. {res['title']}: {res['snippet']}")
            
            elif tool_name == "wikipedia_search":
                if data.get("summary"):
                    info_parts.append(f"위키피디아 정보: {data['title']}")
                    info_parts.append(f"  요약: {data['summary'][:200]}...")
            
            elif tool_name == "get_stock_info":
                info_parts.append(f"주식 정보: {data['company_name']} ({data['symbol']})")
                info_parts.append(f"  현재가: ${data['current_price']}, 변동: {data['change']:+.2f} ({data['change_percent']:+.2f}%)")
                info_parts.append(f"  거래량: {data['volume']:,}")
            
            elif tool_name == "get_weather":
                info_parts.append(f"{data['city']} 날씨 정보:")
                info_parts.append(f"  온도: {data['temperature']}°C, 습도: {data['humidity']}%")
                info_parts.append(f"  날씨: {data['description']}")
            
            elif tool_name == "calculate":
                info_parts.append(f"계산 결과: {data['expression']} = {data['result']}")
        
        aggregated_info = "\n".join(info_parts)
        confidence = min(0.9, 0.6 + len(successful_results) * 0.1)
    
    print(f"✅ 정보 통합 완료 (신뢰도: {confidence:.3f})")
    
    return {
        **state,
        "aggregated_info": aggregated_info,
        "confidence": confidence
    }

def generate_response(state: MCPState) -> MCPState:
    """최종 응답 생성"""
    user_query = state["user_query"]
    aggregated_info = state["aggregated_info"]
    
    print("💬 최종 응답 생성 중...")
    
    # 응답 생성 프롬프트
    prompt = f"""사용자의 질문에 대해 외부 도구들로부터 얻은 정보를 바탕으로 친근하고 유용한 응답을 생성해주세요.

사용자 질문: {user_query}

수집된 정보:
{aggregated_info}

요구사항:
1. 정보를 이해하기 쉽게 정리해서 설명하세요
2. 구체적인 데이터나 수치가 있다면 포함하세요
3. 친근한 톤으로 작성하세요
4. 질문에 직접적으로 답변하세요
5. 정보의 출처를 언급해주세요

응답:"""
    
    final_response = llm.generate(prompt, max_length=400, temperature=0.5)
    
    print(f"📝 응답 생성 완료: {len(final_response)} 글자")
    
    return {
        **state,
        "final_response": final_response
    }

def evaluate_response(state: MCPState) -> MCPState:
    """응답 품질 평가"""
    confidence = state["confidence"]
    errors = state["errors"]
    retry_count = state["retry_count"]
    tool_results = state["tool_results"]
    
    # 재시도 필요 조건
    successful_tools = len([r for r in tool_results if r["success"]])
    need_retry = (
        successful_tools == 0 and      # 성공한 도구가 없고
        retry_count < 1 and            # 재시도 횟수가 적고
        len(errors) > 0                # 오류가 있음
    )
    
    print(f"📊 응답 평가: 신뢰도={confidence:.3f}, 성공도구={successful_tools}, 오류={len(errors)}, 재시도필요={need_retry}")
    
    return {
        **state,
        "need_retry": need_retry
    }

def retry_with_backup(state: MCPState) -> MCPState:
    """백업 전략으로 재시도"""
    print("🔄 백업 전략으로 재시도...")
    
    # 백업 도구로 웹 검색 사용
    return {
        **state,
        "selected_tools": ["web_search"],  # 백업으로 웹 검색만
        "retry_count": state["retry_count"] + 1
    }

def should_retry(state: MCPState) -> str:
    """재시도 여부 결정"""
    if state.get("need_retry", False):
        return "retry"
    else:
        return "end"

print("✅ MCP 워크플로 노드 함수 정의 완료")

## 5. LangGraph MCP 워크플로 구성

In [ ]:
# MCP 워크플로 생성
workflow = StateGraph(MCPState)

# 노드 추가
workflow.add_node("analyze", analyze_intent)
workflow.add_node("prepare", prepare_tool_calls)
workflow.add_node("execute", execute_tools)
workflow.add_node("aggregate", aggregate_information)
workflow.add_node("generate", generate_response)
workflow.add_node("evaluate", evaluate_response)
workflow.add_node("retry", retry_with_backup)

# 엣지 연결
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "prepare")
workflow.add_edge("prepare", "execute")
workflow.add_edge("execute", "aggregate")
workflow.add_edge("aggregate", "generate")
workflow.add_edge("generate", "evaluate")

# 조건부 엣지
workflow.add_conditional_edges(
    "evaluate",
    should_retry,
    {
        "retry": "retry",
        "end": END
    }
)

# 재시도 후 도구 준비로
workflow.add_edge("retry", "prepare")

# 앱 컴파일
mcp_app = workflow.compile()

print("✅ MCP LangGraph 워크플로 구성 완료")

## 6. MCP 시스템 테스트

In [ ]:
def run_mcp_system(query: str):
    """MCP 통합 시스템 실행"""
    print(f"\n🚀 MCP 시스템 시작")
    print(f"❓ 사용자 질문: {query}")
    print("=" * 80)
    
    # 초기 상태
    initial_state = {
        "user_query": query,
        "retry_count": 0,
        "errors": []
    }
    
    # 워크플로 실행
    final_state = mcp_app.invoke(initial_state)
    
    print("\n📊 최종 결과")
    print("=" * 80)
    print(f"🔍 선택된 도구: {final_state['selected_tools']}")
    print(f"⚡ 실행된 도구: {len(final_state['tool_results'])}개")
    print(f"✅ 성공한 도구: {len([r for r in final_state['tool_results'] if r['success']])}개")
    print(f"💬 최종 응답: {final_state['final_response']}")
    print(f"📊 신뢰도: {final_state['confidence']:.3f}")
    print(f"🔄 재시도 횟수: {final_state['retry_count']}")
    
    # 도구 결과 상세 정보
    if final_state['tool_results']:
        print("\n🛠 도구 실행 상세:")
        for i, result in enumerate(final_state['tool_results'], 1):
            status = "✅" if result['success'] else "❌"
            print(f"  [{i}] {status} {result['tool']}")
            if result['success'] and 'result' in result:
                data = result['result']
                if isinstance(data, dict) and 'success' in data:
                    print(f"      데이터: {str(data)[:100]}...")
    
    # 오류 정보
    if final_state['errors']:
        print("\n❌ 발생한 오류:")
        for error in final_state['errors']:
            print(f"  - {error}")
    
    return final_state

# 테스트 질문들
test_queries = [
    "인공지능에 대해 위키피디아에서 검색해줘",
    "애플 주식 가격이 어떻게 되나요?",
    "서울 날씨 알려주세요",
    "123 더하기 456은 얼마인가요?",
    "머신러닝에 대한 정보를 찾아주세요"
]

# 첫 번째 질문으로 테스트
result = run_mcp_system(test_queries[0])

## 7. 다양한 질문으로 종합 테스트

In [ ]:
# 모든 테스트 질문 실행
results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n\n🧪 테스트 {i}/{len(test_queries)}")
    try:
        result = run_mcp_system(query)
        success_rate = len([r for r in result['tool_results'] if r['success']]) / max(len(result['tool_results']), 1) * 100
        
        results.append({
            "query": query,
            "tools_used": result['selected_tools'],
            "success_rate": success_rate,
            "response": result['final_response'],
            "confidence": result['confidence'],
            "retries": result['retry_count'],
            "errors": len(result['errors'])
        })
    except Exception as e:
        print(f"❌ 시스템 오류 발생: {e}")
        results.append({
            "query": query,
            "tools_used": [],
            "success_rate": 0,
            "response": f"시스템 오류: {str(e)}",
            "confidence": 0.0,
            "retries": 0,
            "errors": 1
        })

# 결과 요약
print("\n\n📊 전체 테스트 결과 요약")
print("=" * 100)

avg_success_rate = sum(r["success_rate"] for r in results) / len(results)
avg_confidence = sum(r["confidence"] for r in results) / len(results)
total_retries = sum(r["retries"] for r in results)
total_errors = sum(r["errors"] for r in results)

print(f"평균 도구 성공률: {avg_success_rate:.1f}%")
print(f"평균 신뢰도: {avg_confidence:.3f}")
print(f"총 재시도 횟수: {total_retries}")
print(f"총 오류 수: {total_errors}")

# 도구 사용 통계
tool_usage = {}
for result in results:
    for tool in result["tools_used"]:
        tool_usage[tool] = tool_usage.get(tool, 0) + 1

print("\n🛠 도구 사용 통계:")
for tool, count in sorted(tool_usage.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tool}: {count}회")

print("\n📋 상세 결과:")
for i, result in enumerate(results, 1):
    print(f"\n[{i}] {result['query']}")
    print(f"    사용 도구: {result['tools_used']}")
    print(f"    성공률: {result['success_rate']:.1f}%, 신뢰도: {result['confidence']:.3f}")
    print(f"    응답: {result['response'][:100]}...")

## 8. 고급 기능: 복합 질문 처리

In [ ]:
# 복합 질문들 (여러 도구가 필요한 질문들)
complex_queries = [
    "애플 주식 정보와 서울 날씨를 둘 다 알려주세요",
    "인공지능에 대한 위키피디아 정보를 찾고, 100 곱하기 50도 계산해주세요",
    "테슬라 주식 가격과 부산 날씨 그리고 딥러닝 정보를 모두 알려주세요"
]

print("🔬 복합 질문 테스트")
print("=" * 50)

for i, query in enumerate(complex_queries, 1):
    print(f"\n🧪 복합 질문 {i}")
    try:
        result = run_mcp_system(query)
        tools_count = len(result['selected_tools'])
        success_count = len([r for r in result['tool_results'] if r['success']])
        print(f"✅ 처리 완료: {tools_count}개 도구 선택, {success_count}개 성공")
    except Exception as e:
        print(f"❌ 처리 실패: {e}")

## 9. 성능 분석 및 최적화 제안

In [ ]:
def analyze_mcp_performance():
    """MCP 시스템 성능 분석"""
    print("🔬 MCP 시스템 성능 분석")
    print("=" * 50)
    
    # 전체 성능 지표
    total_queries = len(results)
    high_conf_queries = sum(1 for r in results if r["confidence"] >= 0.7)
    no_error_queries = sum(1 for r in results if r["errors"] == 0)
    
    print(f"전체 질문 수: {total_queries}")
    print(f"고신뢰도 응답: {high_conf_queries}개 ({high_conf_queries/total_queries*100:.1f}%)")
    print(f"오류 없는 응답: {no_error_queries}개 ({no_error_queries/total_queries*100:.1f}%)")
    
    # 도구별 성능 분석
    print("\n🛠 도구별 성능:")
    for tool_name in mcp_registry.list_tools():
        usage_count = tool_usage.get(tool_name, 0)
        if usage_count > 0:
            print(f"  {tool_name}: {usage_count}회 사용")
    
    # 신뢰도 분포
    confidences = [r["confidence"] for r in results]
    high_conf = sum(1 for c in confidences if c >= 0.8)
    medium_conf = sum(1 for c in confidences if 0.5 <= c < 0.8)
    low_conf = sum(1 for c in confidences if c < 0.5)
    
    print("\n📊 신뢰도 분포:")
    print(f"  높음 (≥0.8): {high_conf}개")
    print(f"  보통 (0.5~0.8): {medium_conf}개")
    print(f"  낮음 (<0.5): {low_conf}개")
    
    print("\n🎯 시스템 장점:")
    print("  - 다양한 외부 도구들의 자동 통합")
    print("  - 지능적인 도구 선택 및 매개변수 추출")
    print("  - 다중 도구 결과의 효과적인 통합")
    print("  - 오류 처리 및 백업 전략 제공")
    print("  - Day 1 파인튜닝 모델의 문맥 이해 활용")
    
    print("\n💡 개선 제안:")
    if avg_confidence < 0.7:
        print("  - 더 정교한 의도 분석 및 도구 선택 로직")
        print("  - 도구 결과 검증 및 품질 평가 강화")
    if total_errors > 0:
        print("  - 더 견고한 오류 처리 및 복구 메커니즘")
        print("  - 실제 API 연동 시 속도 제한 및 인증 처리")
    if total_retries > 0:
        print("  - 초기 도구 선택 정확도 향상")
        print("  - 더 나은 백업 전략 개발")
    
    print("\n🚀 확장 가능성:")
    print("  - 실시간 API 연동 (Google Search, OpenWeatherMap 등)")
    print("  - 더 많은 도메인 특화 도구 추가")
    print("  - 사용자 맞춤형 도구 선택 학습")
    print("  - 도구 실행 결과 캐싱 및 최적화")

analyze_mcp_performance()

## 10. 실습 과제

### 과제 1: 새로운 도구 추가
- 환율 정보를 가져오는 도구를 추가해보세요
- 뉴스 검색 도구를 구현해보세요
- 이미지 검색 API 연동을 시도해보세요

### 과제 2: 지능적 도구 선택
- 더 정교한 자연어 처리를 통한 도구 선택 로직을 개선해보세요
- 사용자 질문의 맥락을 더 잘 이해하는 시스템을 구현해보세요

### 과제 3: 결과 통합 개선
- 여러 도구 결과를 더 효과적으로 통합하는 방법을 개발해보세요
- 결과 간의 일관성 검증 메커니즘을 추가해보세요

### 과제 4: 실제 API 연동
- Google Search API나 다른 실제 API와 연동해보세요
- API 키 관리 및 속도 제한 처리를 구현해보세요

In [ ]:
# 정리 및 다음 단계
print("🎓 MCP 통합 실습 완료!")
print("\n다음 단계:")
print("1. 06-gradio-ui.ipynb - Gradio UI 래핑 (최종)")

print("\n💡 핵심 학습 내용:")
print("- MCP(Model Context Protocol) 개념과 구현")
print("- 다양한 외부 도구들의 자동 통합")
print("- 지능적 도구 선택 및 실행 워크플로")
print("- 오류 처리 및 백업 전략")
print("- Day 1 파인튜닝 모델을 통한 맥락 이해")
print("- 복합 질문의 효과적인 처리")

print("\n🌟 실제 활용 사례:")
print("- AI 어시스턴트의 기능 확장")
print("- 멀티모달 정보 검색 시스템")
print("- 비즈니스 인텔리전스 도구 통합")
print("- 자동화된 정보 수집 파이프라인")

print("\n✅ MCP 통합 시스템 구축 완료")